## Enviorment

In [2]:
#verify user
!whoami
#verify user account
!gcloud auth list
#which enciorment/virtualenv running in
import sys
print(sys.prefix)

michael_zl_prime


In [5]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all --purge_orphaned_data True > /tmp/tensorboard_run.log

---

In [6]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import ipywidgets as widgets
import datetime
import tensorflow as tf

from models.six_stage_linear_model import ModelMaker
import callbacks
import dataset_builder
from config import *
import load_weights

print(tf.version.VERSION)

2.0.0


In [ ]:
if TPU_MODE:
    import tpu_training.test_connectivity
    import tpu_training.init_tpu as init_tpu
    strategy=init_tpu.init_tpu()

---
# Make dataset

In [25]:
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
dst=dataset_builder.build_training_ds(tfrecord_files_train)
dsv=dataset_builder.build_validation_ds(tfrecord_files_val)


---
# Callbacks

In [ ]:
fit_callbacks=[
    callbacks.checkpoints_callback
    ,callbacks.tensorboard_callback
    ,callbacks.learning_rate_scheduler_callback
    ,callbacks.print_lr_callback
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [ ]:
#run to clean tensorboard results
#!gsutil -m rm -r {TENSORBOARD_PATH}
#!gsutil -m rm -r {CHECKPOINTS_PATH}



---
# Model
## Load weights

In [9]:
checkpoint,starting_epoch=load_weights.checkpoints_prompt()


Found these checkpoints
0.Dont load checkpoint


In [52]:
model_maker=ModelMaker() #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics={'stage1paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage2paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage3paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage4paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage5heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]    
         ,'stage6heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]
        }
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=tf.keras.losses.MeanSquaredError(name="MSE_loss")
                    ,metrics=metrics                    
                   )
    return train_model,test_model

if TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

---
Actually training

In [53]:
train_history=train_model.fit(
    dst
    ,epochs=TRAINING_EPOCHS
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=callbacks
    ,initial_epoch=starting_epoch
)

Train for 437 steps

Learning rate for epoch 7 is 0.003000000026077032
Epoch 8/10
281/437 [==================>...........] - ETA: 3:25 - loss: 3513169370137980.5000 - stage1paf_output_loss: 1.5843 - stage2paf_output_loss: 1785.4202 - stage3paf_output_loss: 796023.0625 - stage4paf_output_loss: 377732320.0000 - stage5heatmap_output_loss: 162006532096.0000 - stage6heatmap_output_loss: 3513007028043776.0000 - stage1paf_output_mean_absolute_error: 0.0245 - stage2paf_output_mean_absolute_error: 0.6583 - stage3paf_output_mean_absolute_error: 18.2988 - stage4paf_output_mean_absolute_error: 463.9840 - stage5heatmap_output_mean_absolute_error: 12686.9766 - stage6heatmap_output_mean_absolute_error: 1711755.5000
Epoch 00008: saving model to gs://dl_training_results/checkpoints/ModelWeights-01Sun1219-0027-0008.ckpt
281/437 [==================>...........] - ETA: 3:31 - loss: 3513169370137980.5000 - stage1paf_output_loss: 1.5843 - stage2paf_output_loss: 1785.4202 - stage3paf_output_loss: 796023.0625

KeyboardInterrupt: 

In [ ]:
model_path=MODELS_PATH+datetime.datetime.now().strftime("%d(%a)%m%y-%H%M")
tf.keras.models.save_model(train_model,model_path)


In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data